In [3]:
from src.models.video import TimeSformerInferencing
from src.post_processing.utils import read_json, joint_event
from typing import List
inferencing =TimeSformerInferencing(
    num_frames= 30,
    num_classes = 17,
    game_name = 'F:/video_classification/2015-11-07 - 18-00 Manchester United 2 - 0 West Brom',
    image_extractor="fcakyon/timesformer-large-finetuned-k400",
    checkpoint = './ds_tv360_model_timesformer_nepochs_2_nframes_30.pth')

results = inferencing.run(method='from_middle', spacing = 30)
results = "D:\\projects\\v2v\\v5\\2015-11-07 - 18-00 Manchester United 2 - 0 West Brom_prediction.jsonl"
data = read_json(results)

In [13]:
results = "D:\\projects\\v2v\\v5\\2015-11-07 - 18-00 Manchester United 2 - 0 West Brom_prediction.jsonl"

rank = {
    "Goal": 6,
    "Penalty":5,
    "Cards": 4,
    "Foul":3,
    "Substitution":2,
    "Shots":1
}

data = read_json(results)
penalty_check = []
goal_check = []
card_check = []
foul_check = []
sub_check = []
shot_check = []

for item in data:
    if "Penalty" in item['predictions']:
        penalty_check.append(item['chunk_id'])
    if "Goal" in item['predictions']:
        goal_check.append(item['chunk_id'])
    if "Foul" in item['predictions']:
        foul_check.append(item['chunk_id'])
    if "Substitution" in item['predictions']:
        sub_check.append(item['chunk_id'])
    if any(card in item['predictions'] for card in ["Yellow card", "Red card", "Yellow->red card"]):
        card_check.append(item['chunk_id'])
    if any(shot in item['predictions'] for shot in ["Shots on target", "Shots off target"]):
        shot_check.append(item['chunk_id'])

# Convert each list to a set and back to a list to ensure uniqueness
penalty_check = sorted(list(set(penalty_check)))
goal_check = sorted(list(set(goal_check)))
card_check = sorted(list(set(card_check)))
foul_check = sorted(list(set(foul_check)))
sub_check = sorted(list(set(sub_check)))
shot_check = sorted(list(set(shot_check)))

penalty_events = joint_event(penalty_check)
goal_events = joint_event(goal_check)
card_events = joint_event(card_check)
foul_events = joint_event(foul_check)
sub_events = joint_event(sub_check)
shot_events = joint_event(shot_check)

In [5]:
penalty_events, goal_events, card_events, foul_events, sub_events, shot_events

([32, 51, 70, 80, (89, 90)],
 [12, 16, 24, (32, 34), (51, 52), (70, 71), (73, 75), 85, (89, 90)],
 [70],
 [0, 70],
 [0, 29, 35, (58, 59), 60, 70, 75, (78, 80)],
 [16, 21, (26, 28), 40, 51, (71, 73), 84])

In [6]:
def filter_events_by_rank(events_dict, rank_dict):
    # Tạo dictionary để lưu events theo thứ tự rank từ cao xuống thấp
    sorted_events = {
        "Goal": goal_events,
        "Penalty": penalty_events,
        "Cards": card_events,
        "Foul": foul_events,
        "Substitution": sub_events,
        "Shots": shot_events
    }
    
    # Tạo set để lưu các thời điểm đã được assign cho event có rank cao hơn
    assigned_times = set()
    
    # Dictionary để lưu kết quả sau khi lọc
    filtered_events = {event_type: [] for event_type in sorted_events.keys()}
    
    # Xử lý theo thứ tự rank từ cao xuống thấp
    for event_type, _ in sorted(rank_dict.items(), key=lambda x: x[1], reverse=True):
        events = sorted_events[event_type]
        
        for time in events:
            # Xử lý trường hợp time là tuple (khoảng thời gian)
            if isinstance(time, tuple):
                time_range = set(range(time[0], time[1] + 1))
                # Kiểm tra xem khoảng thời gian có trùng với assigned_times không
                if not time_range.intersection(assigned_times):
                    filtered_events[event_type].append(time)
                    assigned_times.update(time_range)
            else:
                # Xử lý trường hợp time là số đơn
                if time not in assigned_times:
                    filtered_events[event_type].append(time)
                    assigned_times.add(time)
    
    return filtered_events

# Dữ liệu đầu vào
penalty_events = [32, 51, 70, 80, (89, 90)]
goal_events = [12, 16, 24, (32, 34), (51, 52), (70, 71), (73, 75), 85, (89, 90)]
card_events = [70]
foul_events = [0, 70]
sub_events = [0, 29, 35, (58, 59), 60, 70, 75, (78, 80)]
shot_events = [16, 21, (26, 28), 40, 51, (71, 73), 84]

rank = {
    "Goal": 6,
    "Penalty": 5,
    "Cards": 4,
    "Foul": 3,
    "Substitution": 2,
    "Shots": 1
}

# Chạy hàm lọc
filtered_results = filter_events_by_rank(None, rank)

# In kết quả
for event_type, events in filtered_results.items():
    print(f"{event_type}: {events}")

Goal: [12, 16, 24, (32, 34), (51, 52), (70, 71), (73, 75), 85, (89, 90)]
Penalty: [80]
Cards: []
Foul: [0]
Substitution: [29, 35, (58, 59), 60]
Shots: [21, (26, 28), 40, 84]


In [12]:
from typing import List, Tuple, Dict, Union, Set
from dataclasses import dataclass

TimeEvent = Union[int, Tuple[int, int]]

@dataclass
class EventFilter:
    """Class để quản lý việc lọc sự kiện theo thứ hạng."""
    
    def __init__(self, rank: Dict[str, int]):
        """
        Khởi tạo EventFilter với rank cho các loại sự kiện.
        
        Args:
            rank: Dict[str, int] - Dictionary chứa thứ hạng của các sự kiện
        """
        self.rank = rank
        self.event_map: Dict[int, str] = {}
        # Danh sách các sự kiện chỉ chấp nhận dạng tuple
        self.tuple_only_events = ["Penalty", "Goal"]
    
    def _expand_time_range(self, event: TimeEvent) -> set:
        """Chuyển đổi một sự kiện thành set các thời điểm."""
        if isinstance(event, tuple):
            return set(range(event[0], event[1] + 1))
        return {event}
    
    def _merge_consecutive_times(self, times: List[int]) -> List[Union[int, Tuple[int, int]]]:
        """
        Gom nhóm các thời điểm liên tiếp thành tuple.
        
        Args:
            times: Danh sách các thời điểm cần gom nhóm
            
        Returns:
            Danh sách các thời điểm đã được gom nhóm
        """
        if not times:
            return []
            
        times.sort()
        result = []
        start = times[0]
        prev = start
        
        for curr in times[1:]:
            if curr != prev + 1:
                if start == prev:
                    result.append(start)
                else:
                    result.append((start, prev))
                start = curr
            prev = curr
            
        # Xử lý phần tử cuối cùng
        if start == prev:
            result.append(start)
        else:
            result.append((start, prev))
            
        return result
    
    def preprocess_events(self, events_dict: Dict[str, List[TimeEvent]]) -> Dict[str, List[TimeEvent]]:
        """
        Tiền xử lý các sự kiện, loại bỏ các sự kiện không phải tuple cho Goal và Penalty.
        
        Args:
            events_dict: Dictionary chứa các danh sách sự kiện
            
        Returns:
            Dictionary đã được tiền xử lý
        """
        preprocessed = {}
        
        for event_type, events in events_dict.items():
            if event_type in self.tuple_only_events:
                # Chỉ giữ lại các sự kiện dạng tuple
                preprocessed[event_type] = [e for e in events if isinstance(e, tuple)]
            else:
                preprocessed[event_type] = events
                
        return preprocessed
    
    def process_events(self, events_dict: Dict[str, List[TimeEvent]]) -> Dict[str, List[TimeEvent]]:
        """
        Xử lý và lọc các sự kiện trùng lặp.
        
        Args:
            events_dict: Dictionary chứa các danh sách sự kiện
            
        Returns:
            Dictionary chứa các sự kiện đã được lọc
        """
        # Tiền xử lý để loại bỏ các sự kiện không phải tuple cho Goal và Penalty
        preprocessed_events = self.preprocess_events(events_dict)
        
        # Sắp xếp các sự kiện theo thứ tự ưu tiên giảm dần
        sorted_events = sorted(
            preprocessed_events.items(),
            key=lambda x: self.rank[x[0]],
            reverse=True
        )
        
        # Xử lý từng loại sự kiện
        for event_type, event_list in sorted_events:
            for event in event_list:
                times = self._expand_time_range(event)
                for time in times:
                    if time not in self.event_map or self.rank[event_type] > self.rank[self.event_map[time]]:
                        self.event_map[time] = event_type
        
        # Tạo kết quả cuối cùng
        result = {event_type: [] for event_type in self.rank.keys()}
        
        # Gom nhóm thời điểm theo loại sự kiện
        event_times: Dict[str, List[int]] = {event_type: [] for event_type in self.rank.keys()}
        for time, event_type in self.event_map.items():
            event_times[event_type].append(time)
        
        # Xử lý từng loại sự kiện
        for event_type, times in event_times.items():
            result[event_type] = self._merge_consecutive_times(times)
            
        return result

# Sử dụng code
if __name__ == "__main__":
    # Dữ liệu đầu vào
    events_dict = {
        "Penalty": [32, 51, 70, 80, (89, 90)],
        "Goal": [12, 16, 24, (32, 34), (51, 52), (70, 71), (73, 75), 85, (89, 90)],
        "Cards": [70],
        "Foul": [0, 70],
        "Substitution": [0, 29, 35, (58, 59), 60, 70, 75, (78, 80)],
        "Shots": [16, 21, (26, 28), 40, 51, (71, 73), 84]
    }

    rank = {
        "Goal": 6,
        "Penalty": 5,
        "Cards": 4,
        "Foul": 3,
        "Substitution": 2,
        "Shots": 1
    }

    # Xử lý sự kiện
    event_filter = EventFilter(rank)
    filtered_events = event_filter.process_events(events_dict)

    # In kết quả
    # for event_type, events in filtered_events.items():
    #     print(f"{event_type}: {events}")
    print(filtered_events)

{'Goal': [(32, 34), (51, 52), (70, 71), (73, 75), (89, 90)], 'Penalty': [], 'Cards': [], 'Foul': [0], 'Substitution': [29, 35, (58, 60), (78, 80)], 'Shots': [16, 21, (26, 28), 40, 72, 84]}


In [9]:
from typing import List, Tuple, Dict, Union, Set
from dataclasses import dataclass

TimeEvent = Union[int, Tuple[int, int]]

@dataclass
class EventFilter:
    """Class để quản lý việc lọc sự kiện theo thứ hạng."""
    
    def __init__(self, rank: Dict[str, int]):
        """
        Khởi tạo EventFilter với rank cho các loại sự kiện.
        
        Args:
            rank: Dict[str, int] - Dictionary chứa thứ hạng của các sự kiện
        """
        self.rank = rank
        self.event_map: Dict[int, str] = {}
        # Danh sách các sự kiện chỉ được phép là tuple
        self.tuple_only_events = ["Penalty", "Goal"]
    
    def _expand_time_range(self, event: TimeEvent) -> set:
        """Chuyển đổi một sự kiện thành set các thời điểm."""
        if isinstance(event, tuple):
            return set(range(event[0], event[1] + 1))
        return {event}
    
    def _merge_consecutive_times(self, times: List[int], force_tuple: bool = False) -> List[Union[int, Tuple[int, int]]]:
        """
        Gom nhóm các thời điểm liên tiếp thành tuple.
        
        Args:
            times: Danh sách các thời điểm cần gom nhóm
            force_tuple: Nếu True, tất cả kết quả sẽ là tuple, kể cả thời điểm đơn lẻ
            
        Returns:
            Danh sách các thời điểm đã được gom nhóm
        """
        if not times:
            return []
            
        times.sort()
        result = []
        start = times[0]
        prev = start
        
        for curr in times[1:]:
            if curr != prev + 1:
                if start == prev and not force_tuple:
                    result.append(start)
                else:
                    result.append((start, prev))
                start = curr
            prev = curr
            
        # Xử lý phần tử cuối cùng
        if start == prev and not force_tuple:
            result.append(start)
        else:
            result.append((start, prev))
            
        return result
    
    def process_events(self, events_dict: Dict[str, List[TimeEvent]]) -> Dict[str, List[TimeEvent]]:
        """
        Xử lý và lọc các sự kiện trùng lặp.
        
        Args:
            events_dict: Dictionary chứa các danh sách sự kiện
            
        Returns:
            Dictionary chứa các sự kiện đã được lọc
        """
        # Sắp xếp các sự kiện theo thứ tự ưu tiên giảm dần
        sorted_events = sorted(
            events_dict.items(),
            key=lambda x: self.rank[x[0]],
            reverse=True
        )
        
        # Xử lý từng loại sự kiện
        for event_type, event_list in sorted_events:
            for event in event_list:
                times = self._expand_time_range(event)
                for time in times:
                    if time not in self.event_map or self.rank[event_type] > self.rank[self.event_map[time]]:
                        self.event_map[time] = event_type
        
        # Tạo kết quả cuối cùng
        result = {event_type: [] for event_type in self.rank.keys()}
        
        # Gom nhóm thời điểm theo loại sự kiện
        event_times: Dict[str, List[int]] = {event_type: [] for event_type in self.rank.keys()}
        for time, event_type in self.event_map.items():
            event_times[event_type].append(time)
        
        # Xử lý từng loại sự kiện
        for event_type, times in event_times.items():
            # Đối với Penalty và Goal, bắt buộc phải là tuple
            force_tuple = event_type in self.tuple_only_events
            result[event_type] = self._merge_consecutive_times(times, force_tuple)
            
        return result
    
    def convert_single_to_tuple(self, event_list: List[TimeEvent]) -> List[Tuple[int, int]]:
        """
        Chuyển đổi tất cả các sự kiện đơn lẻ thành tuple.
        
        Args:
            event_list: Danh sách các sự kiện
            
        Returns:
            Danh sách các tuple
        """
        result = []
        for event in event_list:
            if isinstance(event, tuple):
                result.append(event)
            else:
                result.append((event, event))
        return result

# Sử dụng code
if __name__ == "__main__":
    # Dữ liệu đầu vào
    events_dict = {
        "Penalty": [32, 51, 70, 80, (89, 90)],
        "Goal": [12, 16, 24, (32, 34), (51, 52), (70, 71), (73, 75), 85, (89, 90)],
        "Cards": [70],
        "Foul": [0, 70],
        "Substitution": [0, 29, 35, (58, 59), 60, 70, 75, (78, 80)],
        "Shots": [16, 21, (26, 28), 40, 51, (71, 73), 84]
    }

    rank = {
        "Goal": 6,
        "Penalty": 5,
        "Cards": 4,
        "Foul": 3,
        "Substitution": 2,
        "Shots": 1
    }

    # Xử lý sự kiện
    event_filter = EventFilter(rank)
    filtered_events = event_filter.process_events(events_dict)

    # In kết quả
    for event_type, events in filtered_events.items():
        print(f"{event_type}: {events}")

Goal: [(12, 12), (16, 16), (24, 24), (32, 34), (51, 52), (70, 71), (73, 75), (85, 85), (89, 90)]
Penalty: [(80, 80)]
Cards: []
Foul: [0]
Substitution: [29, 35, (58, 60), (78, 79)]
Shots: [21, (26, 28), 40, 72, 84]


In [20]:
import json
from typing import List, Tuple, Dict, Union, Any, Set
from dataclasses import dataclass
import os

TimeEvent = Union[int, Tuple[int, int]]

def read_json(json_path: str) -> List[Dict[str, Any]]:
    """
    Đọc file JSONL.
    
    Args:
        json_path: Đường dẫn đến file
        
    Returns:
        List các dictionary từ file JSONL
    """
    with open(json_path, encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    return data

def joint_event(one_event_list: List[int]) -> List[Union[int, Tuple[int, int]]]:
    """
    Gom nhóm các thời điểm gần nhau thành tuple.
    
    Args:
        one_event_list: Danh sách các thời điểm
        
    Returns:
        Danh sách các thời điểm đã được gom nhóm
    """
    if not one_event_list:
        return []
        
    close_pairs = []
    i = 0

    while i < len(one_event_list) - 1:
        # Kiểm tra khoảng cách giữa phần tử hiện tại và phần tử tiếp theo
        if one_event_list[i + 1] - one_event_list[i] <= 2:
            close_pairs.append((one_event_list[i], one_event_list[i + 1]))
            i += 2  # Bỏ qua phần tử tiếp theo vì nó đã được ghép cặp
        else:
            close_pairs.append(one_event_list[i])
            i += 1  # Tiếp tục đến phần tử tiếp theo

    # Xử lý phần tử cuối cùng nếu còn lại
    if i == len(one_event_list) - 1:
        close_pairs.append(one_event_list[i])
        
    return close_pairs

@dataclass
class EventProcessor:
    """Class để xử lý và lọc các sự kiện từ file JSON."""
    
    def __init__(self, rank: Dict[str, int]):
        """
        Khởi tạo EventProcessor với rank cho các loại sự kiện.
        
        Args:
            rank: Dict[str, int] - Dictionary chứa thứ hạng của các sự kiện
        """
        self.rank = rank
        self.event_map: Dict[int, str] = {}
        # Danh sách các sự kiện chỉ chấp nhận dạng tuple
        self.tuple_only_events = ["Penalty", "Goal"]
    
    def _expand_time_range(self, event: TimeEvent) -> set:
        """Chuyển đổi một sự kiện thành set các thời điểm."""
        if isinstance(event, tuple):
            return set(range(event[0], event[1] + 1))
        return {event}
    
    def extract_events(self, json_path: str) -> Dict[str, List[TimeEvent]]:
        """
        Trích xuất các sự kiện từ file JSON.
        
        Args:
            json_path: Đường dẫn đến file JSON
            
        Returns:
            Dictionary chứa các danh sách sự kiện
        """
        data = read_json(json_path)
        
        # Khởi tạo các danh sách kiểm tra
        event_checks = {
            "Penalty": [],
            "Goal": [],
            "Cards": [],
            "Foul": [],
            "Substitution": [],
            "Shots": []
        }
        
        # Trích xuất các sự kiện từ dữ liệu
        for item in data:
            if "Penalty" in item.get('predictions', []):
                event_checks["Penalty"].append(item['chunk_id'])
            if "Goal" in item.get('predictions', []):
                event_checks["Goal"].append(item['chunk_id'])
            if "Foul" in item.get('predictions', []):
                event_checks["Foul"].append(item['chunk_id'])
            if "Substitution" in item.get('predictions', []):
                event_checks["Substitution"].append(item['chunk_id'])
            if any(card in item.get('predictions', []) for card in ["Yellow card", "Red card", "Yellow->red card"]):
                event_checks["Cards"].append(item['chunk_id'])
            if any(shot in item.get('predictions', []) for shot in ["Shots on target", "Shots off target"]):
                event_checks["Shots"].append(item['chunk_id'])
        
        # Loại bỏ các phần tử trùng lặp và sắp xếp
        events_dict = {}
        for event_type, check_list in event_checks.items():
            unique_sorted = sorted(list(set(check_list)))
            events_dict[event_type] = joint_event(unique_sorted)
            
            # Đối với Goal và Penalty, chỉ giữ lại các sự kiện dạng tuple
            if event_type in self.tuple_only_events:
                events_dict[event_type] = [e for e in events_dict[event_type] if isinstance(e, tuple)]
        
        return events_dict
    
    def process_events(self, events_dict: Dict[str, List[TimeEvent]]) -> Dict[str, List[TimeEvent]]:
        """
        Xử lý và lọc các sự kiện trùng lặp.
        
        Args:
            events_dict: Dictionary chứa các danh sách sự kiện
            
        Returns:
            Dictionary chứa các sự kiện đã được lọc
        """
        # Sắp xếp các sự kiện theo thứ tự ưu tiên giảm dần
        sorted_events = sorted(
            events_dict.items(),
            key=lambda x: self.rank[x[0]],
            reverse=True
        )
        
        # Xử lý từng loại sự kiện
        for event_type, event_list in sorted_events:
            for event in event_list:
                times = self._expand_time_range(event)
                for time in times:
                    if time not in self.event_map or self.rank[event_type] > self.rank[self.event_map[time]]:
                        self.event_map[time] = event_type
        
        # Tạo kết quả cuối cùng
        result = {event_type: [] for event_type in self.rank.keys()}
        
        # Gom nhóm thời điểm theo loại sự kiện
        event_times: Dict[str, List[int]] = {event_type: [] for event_type in self.rank.keys()}
        for time, event_type in self.event_map.items():
            event_times[event_type].append(time)
        
        # Xử lý từng loại sự kiện
        for event_type, times in event_times.items():
            # Sắp xếp thời điểm trước khi gom nhóm
            sorted_times = sorted(times)
            result[event_type] = joint_event(sorted_times)
            
            # Đối với Goal và Penalty, chỉ giữ lại các sự kiện dạng tuple
            if event_type in self.tuple_only_events:
                result[event_type] = [e for e in result[event_type] if isinstance(e, tuple)]
        
        return result
    
    def process_json(self, json_path: str) -> Dict[str, List[TimeEvent]]:
        """
        Xử lý file JSON để trích xuất và lọc các sự kiện.
        
        Args:
            json_path: Đường dẫn đến file JSON
            
        Returns:
            Dictionary chứa các sự kiện đã được lọc
        """
        events_dict = self.extract_events(json_path)
        return self.process_events(events_dict)


# Sử dụng code
if __name__ == "__main__":
    # Đường dẫn file JSON
    results_path = "D:\\projects\\v2v\\v5\\2015-11-07 - 18-00 Manchester United 2 - 0 West Brom_prediction.jsonl"
    
    # Thứ hạng các sự kiện
    rank = {
        "Goal": 6,
        "Penalty": 5,
        "Cards": 4,
        "Foul": 3,
        "Substitution": 2,
        "Shots": 1
    }
    
    # Xử lý sự kiện
    filtered_events = process_football_events(results_path, rank)
    
    # In kết quả
    for event_type, events in filtered_events.items():
        print(f"{event_type}: {events}")

Goal: [(32, 33), (51, 52), (70, 71), (73, 74), (89, 90)]
Penalty: []
Cards: [80]
Foul: [0]
Substitution: [29, 35, (58, 59), 60, (78, 79)]
Shots: [16, 21, (26, 27), 28, 40, 72, 84]
